# Obtención de los datos

In [3]:
# !pip install lxml
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pyarrow
# !pip install clint

In [1]:
# Importamos librerías
import os
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import requests
import time
import datetime
from distutils.command.build_scripts import first_line_re
import requests
from clint.textui import puts, colored, indent
import re

## SIAP

### Creamos catálogos con los identificadores para la API desde el JSON: Victor-dict.json

In [2]:
# Creamos el folder para los distintos catálogos (uno por filtro)
if not os.path.exists('Catalogos_API_SIAP'):
    os.mkdir("Catalogos_API_SIAP")

# Cargamos el json
df = pd.read_json('SIAP_API_dict.json',orient='index')

# Creamos una lista con los nombres de los filtros y el nombre que tendrán los catálogos
tag = ["municipio","anioagric","cicloProd","modalidad","entidad","cultivo","distrito","mesagric"]
name_csv = ['Municipio','Anio_Agricola','Ciclo_Prod','Modalidad','Entidad','Cultivo','Distrito',"Mes_Agricola"]

# Iteramos sobre los filtros
for i in range(8):
    # Toma los datos pertenecientes a tag
    df_aux2 = df.loc[df.index.str.startswith(tag[i])]
    df_aux = df_aux2.copy()
    # Elimina la str del tag+_ de la columna index
    df_aux.index = df_aux.index.str.replace(tag[i]+'_', "")
    # Convierte el indice en columna
    df_aux.reset_index(inplace=True)
    # Cambia los nombres de las columnas
    df_aux.rename(columns={0: "ID","index":"Name"}, inplace=True)
    # Crea el path donde se guardara
    ruta_completa = os.path.join("Catalogos_API_SIAP",name_csv[i])
    # Guardar el DataFrame en el archivo CSV en la carpeta "Catalogos_API_SIAP"
    df_aux.to_csv(ruta_completa + '.csv', index=False)

#Elimina los df
del df_aux, df_aux2

### Descarga de de los datos

In [3]:
# Creacion del folder Raw
if not os.path.exists('Raw'):
    os.mkdir("Raw")

In [4]:
# Creación del folder SIAP
if not os.path.exists('Raw/SIAP/'):
    os.mkdir("Raw/SIAP/")

In [5]:
# Comando para ignorar warnings (salen en cada request)
warnings.filterwarnings('ignore')

In [6]:
# Creamos iterables para años, meses y cultivos
anio_arr = pd.read_csv('./Catalogos_API_SIAP/Anio_Agricola.csv').ID.to_list()
cultivo_arr = pd.read_csv('./Catalogos_API_SIAP/Cultivo.csv').ID.to_list()
mes_arr = pd.read_csv('./Catalogos_API_SIAP/Mes_Agricola.csv').ID.to_list()

In [8]:
# En el iterable de cultibos eliminamos la opción 0
cultivo_arr.remove(0) # 0: Todos

In [9]:
# En SIAP creamos una carpeta para cada año
for i in anio_arr:
    if not os.path.exists('Raw/SIAP/'+str(i)):
        os.mkdir("Raw/SIAP/"+str(i))

In [10]:
# Función para hacer un request a la API
def API_SIAP(anio,mes,cultivo):
  """Esta función permite hacer un request a la API de avance agrícola mensual y recibe como argumento el año,
     mes y cultivo del que queremos obtener la información. Los demás filtros reciben un valor fijo."""

  # Endpoint
  url = "https://nube.siap.gob.mx/avance_agricola/"

  # Diccionario con atributos del request
  payload = {'xajax': 'reporte', # Para obtener la tabla
  'xajaxr': '1696449941927', # Timestamp UNIX
  'xajaxargs[]': [
      '1', # 1: Desglose por estados, 2: Cultivo total
      str(anio), # Anio
      '5', # ID Ciclo
      '3', # ID Modalidad
      '0', # ID Estado (0: Nacional)
      '--',
      '--',
      str(cultivo), # ID Cultivo
      '200201',
      '0',
      '1', # 1: Por municipio
      'undefined',
      'undefined',
      'undefined',
      str(mes) # Valor del mes: va de 1 a 8, con 1 siendo Enero y 8 Agosto
      ]
  }

  headers = {
    'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360'
  }

  # Hacemos el request
  response = requests.request("POST", url, headers=headers, data=payload, verify=False)
  response.encoding='ISO-8859-1'

  # Regresamos un string con el XML con la tabla que contiene los datos solicitados
  return response.text

In [12]:
# Iteramos anidadamente sobre los años, meses y cultivos de nuestro interés

for anio in anio_arr:     # Iteramos en los años
    for mes in mes_arr:   # Iteramos en los meses

        # Lista con el nombre de nuestras columnas
        columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","Anio", "Mes", "Cultivo"]
        df_raw = pd.DataFrame(columns=columnas)  # Creamos un DataFrame vacío con estas columnas

        for cultivo in cultivo_arr:                 # Iteramos en los cultivos
            xml_data = API_SIAP(anio,mes,cultivo)   # Hcemos el API request
            soup = BeautifulSoup(xml_data, 'lxml')  # Parseamos el XML con BeautifulSoup usando el analizador XML (lxml)

            # Encontramos la tabla por su ID
            tablas = soup.find_all('table', id='Resultados-reporte') #***************************************************************************#

            if len(tablas) == 0:  #**************************************************************************************************************#
                continue          #**************************************************************************************************************#
            
            elif mes in (10, 11, 12):    #**************************************************************************************************************#
                tabla = tablas[-1]       #************************************************************************************************************#
            
            else:                     #**************************************************************************************************************#
                tabla = tablas[0]     #**************************************************************************************************************#
                
            # Inicializamos listas para almacenar los datos
            datos = []

            # Iteramos a través de las filas de la tabla
            for fila in tabla.find_all('tr')[1:]:  # Ignoramos la primera fila de encabezados
                columnas = fila.find_all('td')     # Identificamos todas las celdas
                if len(columnas) == 0:             # Ignoramos las filas vacías
                    continue

                # Extraemos los datos relevantes de cada columna
                #posicion = columnas[0].get_text()
                entidad = columnas[1].get_text()
                municipio = columnas[2].get_text()
                superficie_sembrada = columnas[3].get_text()
                superficie_cosechada = columnas[4].get_text()
                superficie_siniestrada = columnas[5].get_text()
                produccion = columnas[6].get_text()
                rendimiento = columnas[7].get_text()

                # Agregamos los datos a la lista
                datos = [
                    entidad, municipio,
                    superficie_sembrada, superficie_cosechada,
                    superficie_siniestrada, produccion, rendimiento,
                    anio, mes, cultivo
                ]

                # Agregamos estos datos al DataFrame
                df_raw.loc[len(df_raw)] = datos

        # Guardamos un CSV cuyo nombre sigue el formato: "Avance_agricola_año_mes.csv"
        name_csv = 'Avance_Agricola_'+str(anio)+'_'+str(mes)
        df_raw.to_csv('./Raw/SIAP/'+str(anio)+'/'+name_csv+'.csv', index=False)

## SNIIM

In [13]:
# Creacion de folder en raw
if not os.path.exists('raw/SNIIM'):
    os.mkdir("raw/SNIIM")

### Descarga de datos

In [4]:
class ScrapperMarketAgriculture:
    total_records = 0
    inserted_records = 0
    current_product = 'None'
    first_print = True

    base_url = 'http://www.economia-sniim.gob.mx/NUEVO/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas'
    init_urls = [
        ['Frutas y Hortalizas', '/ConsultaFrutasYHortalizas.aspx', '/ResultadosConsultaFechaFrutasYHortalizas.aspx'],
        ['Flores', '/ConsultaFlores.aspx?SubOpcion=5', '/ResultadosConsultaFechaFlores.aspx'],
        ['Granos', '/ConsultaGranos.aspx?SubOpcion=6', '/ResultadosConsultaFechaGranos.aspx'],
        ['Aceites', '/ConsultaAceites.aspx?SubOpcion=8', '/ResultadosConsultaFechaAceites.aspx']
    ]

    last_year = 2023

    def __init__(self, *args, **kwargs):
        self.is_historic = kwargs.get('is_historic', True)
        self.df = pd.DataFrame()

    def read_category(self, category, url, url_form):
        category_page = requests.get(self.base_url + url)
        category_page = BeautifulSoup(category_page.content, features="html.parser")

        products = [(product.getText(), product['value'], ) for product in category_page.select_one('select#ddlProducto').find_all('option')]

        for product in products[105:]:
            product_name, product_id = product
            if product_id == '-1':
                continue

            if self.current_product != 'None':
              prod = re.sub(r'\s','_',self.current_product)
              self.df.to_csv(f'./raw/SNIIM/sniim_product_{prod}.csv', index=False)
            self.current_product = str(product_name).lower().replace('-','').replace('  ', '_')
            self.first_print = True
            self.df = pd.DataFrame()

            with indent(4):
                puts(colored.magenta("Producto: {}".format(self.current_product)))

            if self.is_historic:
                for year in range(2018, 2024):
                    payload = {
                        'fechaInicio':'01/01/{0}'.format(str(year)),
                        'fechaFinal':'01/01/{0}'.format(str(year + 1)),
                        'ProductoId':product_id,
                        'OrigenId':'-1',
                        'Origen':'Todos',
                        'DestinoId':'-1',
                        'Destino':'Todos',
                        'PreciosPorId':'2',
                        'RegistrosPorPagina':'1000'
                    }

                    if not self.gather_prices(payload, url_form):
                        continue
            else:
                today = datetime.datetime.today()
                deleta = datetime.timedelta(days=-1)
                payload = {
                        'fechaInicio':'{}'.format(today.strftime('%d/%m/%Y')),
                        'fechaFinal':'{}'.format((today).strftime('%d/%m/%Y')),
                        'ProductoId':product_id,
                        'OrigenId':'-1',
                        'Origen':'Todos',
                        'DestinoId':'-1',
                        'Destino':'Todos',
                        'PreciosPorId':'2',
                        'RegistrosPorPagina':'1000'
                    }

                if not self.gather_prices(payload, url_form):
                    continue

        return

    def scraping(self):
        self.total_records = 0
        self.inserted_records = 0

        for category, url, url_form in self.init_urls:
            self.read_category(category, url, url_form)
            time.sleep(60)

    def gather_prices(self, payload, url_form):
        with indent(4):
            puts(colored.blue("Peticion: {}".format(str(payload))))

        response = requests.get(self.base_url + url_form, params=payload)
        time.sleep(30)
        if response.status_code != 200:
            with indent(4):
                puts(colored.red("Error en la peticion HTTP: {}".format(str(response.text))))
            return False

        product_prices = BeautifulSoup(response.content, features="html.parser")

        try:
            table_prices = product_prices.select_one('table#tblResultados')
        except Exception as error:
            with indent(4):
                puts(colored.red("Error en el parseo: {}".format(str(error))))
            return False

        fields = ('fecha', 'presentacion', 'origen', 'destino', 'precio_min', 'precio_max', 'precio_frec', 'obs')
        counter_row = 0

        for observation in table_prices.find_all('tr'):
            if counter_row > 1:
                row = {}
                counter_field = 0
                if self.first_print:
                  self.first_print = False
                for metric in observation.find_all('td'):
                    row[fields[counter_field]] = metric.getText()
                    counter_field += 1

                row['name'] = self.current_product
                df2 = pd.DataFrame(row, index=[0]) 
                self.df = pd.concat([self.df, df2])

            self.total_records += 1
            counter_row += 1
            if self.total_records % 1000 == 0:
              prod = re.sub(r'\s','_',self.current_product)
              self.df.to_csv(f'./raw/SNIIM/sniim_product_{prod}.csv', index=False) 
        return True



In [6]:
if __name__ == '__main__':
  agricola = ScrapperMarketAgriculture()
  agricola.scraping()